# Source Location Tracking

This tutorial shows how t-prompts automatically tracks where each prompt was created in your source code.

Source location tracking captures:
- **Filename** - which file the prompt was created in
- **File path** - the full absolute path
- **Line number** - which line in the file

This is invaluable for debugging complex prompt hierarchies and maintaining provenance.

In [1]:
from t_prompts import prompt

## Automatic Source Location Capture

Every prompt automatically captures its source location - no configuration needed!

In [2]:
# This prompt will remember it was created on this specific line
task = "translate to French"
p = prompt(t"Task: {task:t}")

# Access source location from the interpolation
loc = p['t'].source_location

print("This prompt was created at:")
print(f"  File: {loc.filename}")
print(f"  Path: {loc.filepath}")
print(f"  Line: {loc.line}")
print(f"\nFormatted: {loc.format_location()}")

This prompt was created at:
  File: 3863425204.py
  Path: /private/var/folders/wr/5n6v6bvj6mxdxjcg5_5ss1ph0000gn/T/ipykernel_26667/3863425204.py
  Line: 3

Formatted: 3863425204.py:3


## Checking Availability

Source location might not always be available (e.g., in some REPL contexts). Check before using:

In [3]:
context = "User is a beginner"
p = prompt(t"Context: {context:ctx}")

loc = p['ctx'].source_location

if loc and loc.is_available:
    print(f"✓ Location available: {loc.format_location()}")
else:
    print("✗ Source location not available")

✓ Location available: 163145370.py:2


## Source Location for All Elements

Both static text and interpolations capture source location information.

In [4]:
from t_prompts import Static, TextInterpolation

name = "Alice"
p = prompt(t"Hello {name:n}!")

print("Source locations for all elements:\n")
for i, elem in enumerate(p.children):
    loc = elem.source_location

    if isinstance(elem, Static):
        content = f"Static: {elem.value!r}"
    elif isinstance(elem, TextInterpolation):
        content = f"Interpolation '{elem.key}': {elem.value!r}"
    else:
        content = f"Element {i}"

    print(content)
    if loc and loc.is_available:
        print(f"  → {loc.format_location()}")
    print()

Source locations for all elements:

Static: 'Hello '
  → 3995065992.py:4

Interpolation 'n': 'Alice'
  → 3995065992.py:4

Static: '!'
  → 3995065992.py:4



## Nested Prompts Have Independent Locations

Each prompt remembers where it was created, even when nested inside other prompts.

In [5]:
# Create inner prompt on this line
greeting = "Hello"
inner = prompt(t"{greeting:greet}, world!")
inner_line = inner['greet'].source_location.line if inner['greet'].source_location else None

# Create outer prompt on a different line
outer = prompt(t"Message: {inner:msg}")
outer_line = outer['msg'].source_location.line if outer['msg'].source_location else None

print(f"Inner prompt created at line {inner_line}")
print(f"Outer prompt created at line {outer_line}")
if inner_line and outer_line:
    print(f"\n✓ Different lines: {inner_line != outer_line}")

Inner prompt created at line 3
Outer prompt created at line 7

✓ Different lines: True


## Accessing Location from Static Elements

Static text elements also have source location tracking.

In [6]:
p = prompt(t"Static text only")

# Get the first static element
first_elem = p.children[0]
loc = first_elem.source_location

print(f"Element type: {type(first_elem).__name__}")
print(f"Content: {first_elem.value!r}")
if loc and loc.is_available:
    print(f"Created at: {loc.format_location()}")

Element type: Static
Content: 'Static text only'
Created at: 1102225522.py:1


## Source Location in JSON Export

Source location information is included when exporting prompts to JSON.

In [7]:
import json

task = "analyze sentiment"
p = prompt(t"Task: {task:t}")

# Export to JSON
json_data = p.toJSON()

# Pretty print the JSON to see source location
print("JSON export includes source location:")
print(json.dumps(json_data, indent=2))

JSON export includes source location:
{
  "prompt_id": "db4e4a05-6cc4-4522-90b4-933572431787",
  "children": [
    {
      "type": "static",
      "id": "49719d5d-77f1-4276-a03e-a92fb1890bb8",
      "parent_id": "db4e4a05-6cc4-4522-90b4-933572431787",
      "key": 0,
      "index": 0,
      "source_location": {
        "filename": "3408044115.py",
        "filepath": "/private/var/folders/wr/5n6v6bvj6mxdxjcg5_5ss1ph0000gn/T/ipykernel_26667/3408044115.py",
        "line": 4
      },
      "value": "Task: "
    },
    {
      "type": "interpolation",
      "id": "e3b74ae2-02c7-4008-872a-06dd577275c8",
      "parent_id": "db4e4a05-6cc4-4522-90b4-933572431787",
      "key": "t",
      "index": 1,
      "source_location": {
        "filename": "3408044115.py",
        "filepath": "/private/var/folders/wr/5n6v6bvj6mxdxjcg5_5ss1ph0000gn/T/ipykernel_26667/3408044115.py",
        "line": 4
      },
      "expression": "task",
      "conversion": null,
      "format_spec": "t",
      "render_hin

## Disabling Source Location Tracking

You can disable source location capture for performance-critical code.

In [8]:
# Disable source location tracking
data = "some value"
p = prompt(t"Data: {data:d}", capture_source_location=False)

loc = p['d'].source_location
print(f"Source location when disabled: {loc}")
print(f"Is available: {loc.is_available if loc else False}")

Source location when disabled: None
Is available: False


## Use Case: Debugging Complex Prompt Hierarchies

Source location helps you trace where each component of a complex prompt was created.

In [9]:
def create_system_prompt():
    """Creates a system prompt."""
    role = "helpful assistant"
    return prompt(t"You are a {role:role}.")

def create_user_message(task):
    """Creates a user message."""
    return prompt(t"Task: {task:task}")

def build_conversation(task_description):
    """Builds a complete conversation prompt."""
    system = create_system_prompt()
    user = create_user_message(task_description)
    return prompt(t"System: {system:sys}\nUser: {user:usr}")

# Build a conversation
conv = build_conversation("translate to French")

# Trace where each component originated
print("Prompt component origins:\n")

sys_loc = conv['sys'].source_location
if sys_loc and sys_loc.is_available:
    print(f"System prompt: {sys_loc.filename}:{sys_loc.line}")

usr_loc = conv['usr'].source_location
if usr_loc and usr_loc.is_available:
    print(f"User message: {usr_loc.filename}:{usr_loc.line}")

Prompt component origins:

System prompt: 2151340285.py:14
User message: 2151340285.py:14


## Use Case: Audit Logging

Track where prompts were created for compliance and debugging purposes.

In [10]:
def log_prompt_creation(p, prompt_name):
    """Log where a prompt was created for audit purposes."""
    # Get location from the prompt's first element
    if p.children:
        loc = p.children[0].source_location
        if loc and loc.is_available:
            print(f"[AUDIT] Prompt '{prompt_name}' created at {loc.filepath}:{loc.line}")
            return
    print(f"[AUDIT] Prompt '{prompt_name}' created (location unavailable)")

# Example: logging creation of a sensitive prompt
user_email = "user@example.com"
p = prompt(t"Email: {user_email:email}")
log_prompt_creation(p, "user_email_prompt")

# Another example
api_key = "sk-abc123"
p2 = prompt(t"API Key: {api_key:key}")
log_prompt_creation(p2, "api_key_prompt")

[AUDIT] Prompt 'user_email_prompt' created at /private/var/folders/wr/5n6v6bvj6mxdxjcg5_5ss1ph0000gn/T/ipykernel_26667/2422407808.py:13
[AUDIT] Prompt 'api_key_prompt' created at /private/var/folders/wr/5n6v6bvj6mxdxjcg5_5ss1ph0000gn/T/ipykernel_26667/2422407808.py:18


## Use Case: Error Messages with Context

Provide better error messages by showing where prompts were created.

In [11]:
def validate_prompt_length(p, max_length, name):
    """Validate prompt length and show source location on error."""
    text = str(p)
    if len(text) > max_length:
        # Get source location for better error message
        if p.children:
            loc = p.children[0].source_location
            if loc and loc.is_available:
                location_info = f" (created at {loc.format_location()})"
            else:
                location_info = ""
        else:
            location_info = ""

        print(f"ERROR: Prompt '{name}' exceeds max length of {max_length}{location_info}")
        print(f"  Actual length: {len(text)}")
        return False
    return True

# Test with a short prompt
short = prompt(t"Short message")
validate_prompt_length(short, max_length=100, name="short_prompt")

# Test with a long prompt
long_text = "This is a very long prompt that exceeds our maximum allowed length for demonstration purposes."
long = prompt(t"{long_text:text}")
validate_prompt_length(long, max_length=50, name="long_prompt")

ERROR: Prompt 'long_prompt' exceeds max length of 50 (created at 2289560413.py:26)
  Actual length: 94


False

## SourceLocation API Reference

The `SourceLocation` dataclass provides the following:

### Fields

- **`filename`** (str | None): Short filename (e.g., `'script.py'`)
- **`filepath`** (str | None): Full absolute path to the file
- **`line`** (int | None): Line number where prompt was created (1-indexed)

### Properties

- **`is_available`**: Returns `True` if location information is present

### Methods

- **`format_location()`**: Returns formatted string like `"script.py:42"`

In [12]:
# Demonstrate the API
msg = "test"
p = prompt(t"Message: {msg:m}")
loc = p['m'].source_location

print("SourceLocation fields:")
print(f"  filename: {loc.filename}")
print(f"  filepath: {loc.filepath}")
print(f"  line: {loc.line}")
print("\nProperties:")
print(f"  is_available: {loc.is_available}")
print("\nMethods:")
print(f"  format_location(): {loc.format_location()}")

SourceLocation fields:
  filename: 1708099841.py
  filepath: /private/var/folders/wr/5n6v6bvj6mxdxjcg5_5ss1ph0000gn/T/ipykernel_26667/1708099841.py
  line: 3

Properties:
  is_available: True

Methods:
  format_location(): 1708099841.py:3


## Summary

Source location tracking provides automatic provenance for debugging and auditing:

✅ **Automatic** - Captured by default for all prompts  
✅ **Lightweight** - Uses only stack frame info, no file I/O  
✅ **Comprehensive** - Tracks static text, interpolations, and nested prompts  
✅ **Exportable** - Included in `toJSON()` output  
✅ **Optional** - Can be disabled with `capture_source_location=False`  

This feature makes debugging complex prompt systems significantly easier by showing you exactly where each component was created.